In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import set_option
import time
from numpy import set_printoptions
from numpy.random import random, uniform, choice
from numpy import set_printoptions, int0
from numpy import (zeros, min, log10, tanh, arctanh,
                    shape, clip, exp, pi, cos,sqrt, ravel, log)
from copy import deepcopy
import typing as t
from sklearn.linear_model import LinearRegression
from typing import Any

# <div style="color:blue">MULTIPLE REGRESSION</div>

In [2]:
# File Paths 
compression = "/home/adisa/Desktop/Working/Akinwale/compression.csv"
flexural = "/home/adisa/Desktop/Working/Akinwale/flexural.csv"
split = "/home/adisa/Desktop/Working/Akinwale/split.csv"


In [3]:
def linear_regression(path: str) -> Any:
    p = path.split("/")[-1].split(".")[0]
    df_com = pd.read_csv(path)
    df_des= df_com.describe()
    lb =  df_des.iloc[3, :-1].values
    ub = df_des.iloc[-1,:-1].values
    X = df_com.iloc[:, 1:-1].values
    model_x = df_com.iloc[::9, 1: -1].values
    Y = df_com.iloc[:, -1].values
    model = LinearRegression().fit(X, Y)
    r_sq = model.score(X, Y)
    inter = model.intercept_
    coeff = model.coef_
    print(p)
    statement = f"b0:{round(inter, 2)}, b1:{round(coeff[0], 2)}, b2:{round(coeff[1], 2)}, b3:{round(coeff[2], 2)}, b4:{round(coeff[3], 2)}, b5:{round(coeff[4], 2)}"
    return X, lb, ub

In [4]:
###################
# Compression, flexural and split
lr_com = linear_regression(compression)
lr_flex = linear_regression(flexural)
lr_split = linear_regression(split)

compression
flexural
split


In [5]:
xc, lb_c, ub_c = lr_com
xf, lb_f, ub_f = lr_flex
xs, lb_s, ub_s = lr_split

# <div style="color:green">Memory</div>

In [6]:
def memory(pol_size, d):
    lb = [0.113, 0, 0.457, 1.371, 0.113]
    ub = [1.029, 0.206, 2.286, 6.857, 0.566]
    Matrix_Harmony_Memory = np.zeros((pol_size,d))
    #print(Matrix_Harmony_Memory)
    for i in range(pol_size):
        for j in range(d):
            Matrix_Harmony_Memory[i,j] =lb[j] + np.random.random()*(ub[j]-lb[j])
    return Matrix_Harmony_Memory

In [7]:
X_memory = memory(1000, 5)
lb = [0.113, 0, 0.457, 1.371, 0.113]
ub = [1.029, 0.206, 2.286, 6.857, 0.566]

# <div style="color:green">META_HEURISTIC ALGORITHMS</div>

## <div style="color:yellow">CONSTANTS</div>


In [8]:
# harmony      
HMCR = 0.95                   #Harmony memory consideration rate
PAR = 0.3                     #Pitch Adjustment rate
bw = 1                        #bandwidth
# Fire-fly
ALPHA = 1 # randomness 
GAMMA = 0.01 #absorption coefficient
BETA0 = 1 
THETA= 0.97
LOLP = 0.1 # adjustable
# slime mould
EPSILON = 10E-10
ID_FIT = 1                   #id of fittness 
ID_WEI = 2                   #id of weight
ID_POS = 0                   # id of best position normal
ID_BEST_F= 0                 # best position fittness
ID_WORST_F = -1              # worst position
Z = 0.03
# general
num_iter = 1000 #number of iterations
runs = 5

# <div style="color:grey">OBJECTIVE FUNCTIONS</div>


In [9]:
from math import ceil
def obj_func_compression(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    comp = 34.81+409.54*ce+648.19*sba+0*fi-34.15*co+1.07*wa
    return comp

def obj_func_flexural(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    flexural = 19.49-0.14*ce-8.84*sba+0.6*fi-1.19*co+0.15*wa
    return flexural

def obj_func_split(x):
    x = np.array(x).tolist()
    ce, sba, fi, co, wa = x
    split = 54.88-4.28*ce-320.83*sba+0*fi-6*co+0*wa
    return split
def generic_func(x):
     x = np.array(x).tolist()
     ce, sba, fi, co, wa = x
     res = abs(34.81+409.54*ce+648.19*sba+0*fi-34.15*co+1.07*wa)+ \
            abs(19.49-0.14*ce-8.84*sba+0.6*fi-1.19*co+0.15*wa)+ \
            abs(54.88-4.28*ce-320.83*sba+0*fi-6*co+0*wa)
     return res

# <div style="color:grey">SLIME MOULD</div>


In [10]:
def slime_mould_search(lb: list, ub: list, memory: np.array, obj_func: t.Callable):
    start = time.time()
    global_best_score = {}
    pol_size = len(memory)
    d = len(lb)
    def create_solution (x):
        # position = (uniform(lb, ub)) #position
        position = x
        fittness = obj_func(position)#fitness
        weight = zeros(d) #weight
        return [position, fittness, weight]
    for run in range(runs):

        pol = [create_solution(memory[_, :]) for _ in range(pol_size)]
        pol = sorted(pol, key = lambda fit : fit[ID_FIT])
        global_best_init = deepcopy(pol[ID_BEST_F])

        best_score=list()
        components = list()
        times = list()
        for iter in range(num_iter):
            s = pol[0][ID_FIT] - pol[-1][ID_FIT] + EPSILON  #bf and wf === best fitness and the weak fitness              
            # calculate the fitness weight of each slime mold
            for i in range(0, pol_size):
                # Eq.(2.5)
                if i <= int(pol_size / 2):
                    r = uniform(0, 1, d)
                    u = pol[0][ID_FIT] - pol[i][ID_FIT]
                    pol[i][ID_WEI] = 1 +  r* log10((u/ s) + 1)
                else:
                    r = uniform(0, 1, d)
                    u = pol[0][ID_FIT] - pol[i][ID_FIT]
                    pol[i][ID_WEI] = 1 -  r * log10((u/ s) + 1)

            a = arctanh(-((iter + 1) / num_iter) + 1)                   # Eq.(2.4)
            b = 1 - (iter + 1) / num_iter

            # Update the Position of search agents
            for i in range(0, pol_size):
                if uniform() < Z:                                          # Eq.(2.7)
                    pol[i][ID_POS] = uniform(lb, ub)
                else:
                    p = tanh(abs(pol[i][ID_FIT] - global_best_init[ID_FIT]))    # Eq.(2.2)
                    vb = uniform(-a, a, d)                      # Eq.(2.3)
                    vc = uniform(-b, b, d)
                    for j in range(0, d): #j each of the components of slime mould
                        # two positions randomly selected from polulation
                        id_a, id_b = choice(list(set(range(0, pol_size)) - {i}), 2, replace=False)
                        if uniform() < p:  # Eq.(2.1)
                            pol[i][ID_POS][j] = global_best_init[ID_POS][j] + vb[j] * (
                                        pol[i][ID_WEI][j] * pol[id_a][ID_POS][j] - pol[id_b][ID_POS][j])
                        else:
                            pol[i][ID_POS][j] = vc[j] * pol[i][ID_POS][j]

            for i in range(0, pol_size):
                position_new = clip(pol[i][ID_POS], lb, ub)
                fittness_new = obj_func(position_new)
                pol[i][ID_POS] = position_new
                pol[i][ID_FIT] = fittness_new
                
            pol= sorted(pol, key=lambda temp: temp[ID_FIT]) #sorted polpulation
            global_best_new = deepcopy(pol[ID_BEST_F])            
            
            print(f'Iteration {iter} ==> Positions: {global_best_new[0]}, \
                Fitness: {global_best_new[1]} and Weights: {global_best_new[2]}')
            
            best_score.append(global_best_new[1])
            components.append(global_best_new[0])
            times.append(time.time()  - start)

        global_best_score[f'best_score_run_{run}'] = best_score
        global_best_score[f'components_run_{run}'] = components
        global_best_score[f'times_run_{run}'] = times

    
    return global_best_score, components, global_best_score

In [1]:
### General
gen_res = slime_mould_search(lb,ub, X_memory, generic_func)

NameError: name 'slime_mould_search' is not defined

In [ ]:
compression_slime_search = slime_mould_search(lb_c, ub_c, xc, obj_func_compression)

Iteration 0 ==> Positions: [0.11314288 0.         0.45714289 1.37142889 0.11314289],                 Fitness: 34.43330277588871 and Weights: [1. 1. 1. 1. 1.]
Iteration 1 ==> Positions: [0.11314288 0.         0.45714289 1.37142889 0.11314289],                 Fitness: 34.43330277588871 and Weights: [1. 1. 1. 1. 1.]
Iteration 2 ==> Positions: [0.11314288 0.         0.45714289 1.37142889 0.11314289],                 Fitness: 34.43330277588871 and Weights: [1.03756694 1.0736464  1.05311784 1.01398342 1.01929191]
Iteration 3 ==> Positions: [0.11314288 0.         0.45714289 1.37142889 0.11314289],                 Fitness: 34.43330277588871 and Weights: [1. 1. 1. 1. 1.]
Iteration 4 ==> Positions: [0.11314288 0.         0.45714289 1.37142889 0.11314289],                 Fitness: 34.43330277588871 and Weights: [1.00189467 1.01725716 1.00888357 1.01964161 1.00670639]
Iteration 5 ==> Positions: [0.11314288 0.         0.45714289 1.37142889 0.11314289],                 Fitness: 34.43330277588871 an

In [ ]:
split_slime_search = slime_mould_search(lb_s, ub_s, xs, obj_func_split)

Iteration 0 ==> Positions: [0.32091433 0.06418287 0.71314289 2.13942889 0.17650289],                 Fitness: 20.078124207333225 and Weights: [1.07794527 1.00511654 1.00393184 1.01489708 1.03026301]
Iteration 1 ==> Positions: [0.32091433 0.06418287 0.71314289 2.13942889 0.17650289],                 Fitness: 20.078124207333225 and Weights: [1.00009556 1.0045924  1.04343167 1.02581056 1.03520416]
Iteration 2 ==> Positions: [0.32091433 0.06418287 0.71314289 2.13942889 0.17650289],                 Fitness: 20.078124207333225 and Weights: [1.07704586 1.03251673 1.03833431 1.09089343 1.02294144]
Iteration 3 ==> Positions: [0.32091433 0.06418287 0.71314289 2.13942889 0.17650289],                 Fitness: 20.078124207333225 and Weights: [0.92970159 0.84062657 0.99044098 0.94647508 0.81911905]
Iteration 4 ==> Positions: [0.32091433 0.06418287 0.71314289 2.13942889 0.17650289],                 Fitness: 20.078124207333225 and Weights: [0.84789647 0.96987195 0.91046541 0.81307171 0.78739439]
Itera

In [ ]:
flexural_slime_search = slime_mould_search(lb_f, ub_f, xf, obj_func_flexural)

Iteration 0 ==> Positions: [1.02857144 0.20571429 2.28571444 6.85714333 0.56571433],                 Fitness: 10.823770933999999 and Weights: [0.90802936 0.91768518 0.85020738 0.82924493 0.86734154]
Iteration 1 ==> Positions: [1.02857144 0.20571429 2.28571444 6.85714333 0.56571433],                 Fitness: 10.823770933999999 and Weights: [1.04201189 1.07944671 1.02882224 1.02278532 1.13590614]
Iteration 2 ==> Positions: [1.02857144 0.20571429 2.28571444 6.85714333 0.56571433],                 Fitness: 10.823770933999999 and Weights: [0.81236564 0.71287883 0.82427667 0.82039626 0.94628816]
Iteration 3 ==> Positions: [1.02857144 0.20571429 2.28571444 6.85714333 0.56571433],                 Fitness: 10.823770933999999 and Weights: [0.76140204 0.92785956 0.90598364 0.93769887 0.77223685]
Iteration 4 ==> Positions: [1.02857144 0.20571429 2.28571444 6.85714333 0.56571433],                 Fitness: 10.823770933999999 and Weights: [0.94374645 0.77259434 0.91786558 0.81403483 0.94197188]
Itera

In [ ]:
flexural_slime_search

({'best_score_run_0': [10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.859902592258216,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.880326047906468,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.877582657513887,
   10.823770933999999,
   10.86910792864149,
   10.823770933999999,
   10.823770933999999,
   10.850039098761037,
   10.823770933999999,
   10.823770933999999,
   10.82377948547084,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.823770933999999,
   10.8237709339